# Interactive visualization of pores varying $C_{min}$ and $K_{max}$ parameters of binarization

This is the predecessor of the online tool to collect the best parameters for binarization using $C_{min}$ and $K_{max}$ as parameters.

This notebook is intended to be run step-by-step because it has multiple interactive cells. Or if your notebook IDE supports, you can run all cells up to the interactive cell of interest.

In [ ]:
import os
print(os.getcwd())

In [ ]:
from pre_sal_ii.improc import colorspace
from pre_sal_ii.improc import scale_image_and_save

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
image_name = "ML-tste_original_test1"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)


In [ ]:
%matplotlib inline
image_name = "ML-tste_original_test1"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage = cv2.imread(path)
plt.close("all")
plt.imshow(inputImage[:,:,::-1])
plt.show()

# BGR to CMKY:
inputImageCMYK = colorspace.bgr2cmyk(inputImage)

In [ ]:
binaryImage = cv2.inRange(
    inputImageCMYK,
    (60,   0,   0,   0),
    (255, 255,  64, 196))
binaryImage

In [ ]:
%matplotlib inline
plt.close("all")
plt.imshow(binaryImage, cmap='gray')
plt.show()

In [ ]:
from skimage.measure import label, regionprops

label_img = label(binaryImage)
regions = regionprops(label_img)

In [ ]:
def regions_df(regions, label_img):
    all_objs = []
    for it, region in enumerate(regions):
        ys = (region.coords.T[0] - label_img.shape[0]/2)/(label_img.shape[0]/2)
        xs = (region.coords.T[1] - label_img.shape[1]/2)/(label_img.shape[1]/2)
        obj = {
            "area": region.area,
            "max-dist": max((ys**2 + xs**2)**0.5),
        }
        all_objs.append(obj)

    df = pd.DataFrame(all_objs)
    return df

df = regions_df(regions, label_img)
df

In [ ]:
sum(df["area"] > 400)

In [ ]:
max_dist = max(df["max-dist"])
colored_image3 = np.zeros((*label_img.shape, 3), dtype=np.uint8)
for it, region in enumerate(regions):
    if df["area"].iloc[it] > 400:
        color_value = np.random.randint(64, 255, size=3)
        colored_image3[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
%matplotlib inline
plt.close("all")
plt.imshow(colored_image3)
plt.show()

In [ ]:
len(range(0, 256, 16))

In [ ]:
def rmm(value, minvalue, maxvalue):
    value = min(value, maxvalue)
    value = max(value, minvalue)
    return value

rmm(3, 0, 2)

In [ ]:
def get_images():

    thresh = 480
    step = 4

    map_img = np.zeros((256//step, 256//step, 3), dtype=np.uint8)

    colors = [
        np.array([0, 0, 0]),
        np.array([0, 0, 255]),
        np.array([255, 0, 0]),
        np.array([0, 255, 0]),
        np.array([255, 255, 255]),
        ]

    from tqdm import tqdm

    images = [[None for _ in range(256//step)] for _ in range(256//step)]

    for c in tqdm([*range(0, 256, step)]):
        for k in range(0, 256, step):
            binaryImage = cv2.inRange(
                inputImageCMYK,
                ( c,   0,   0,   0),
                (255, 255,  64, k))
                #(255, 255,  64, 196))

            label_img = label(binaryImage)
            regions = regionprops(label_img)
            df = regions_df(regions, label_img)
            
            bin_image_4 = np.zeros((*label_img.shape, 3), dtype=np.uint8)
            id_region = 0
            for it, region in enumerate(regions):
                #display(df)
                if df["area"].iloc[it] > thresh:
                    id_region += 1
                    #color_value = np.array([255, 255, 255])
                    color_value = colors[rmm(id_region, 0, len(colors)-1)]
                    bin_image_4[region.coords.T[0], region.coords.T[1]] = color_value
            images[c//step][k//step] = bin_image_4
            
            # plt.imshow(binaryImage, cmap='gray')
            # plt.show()
            
            if len(regions) == 0:
                count_regions = 0
            else:
                count_regions = sum(df["area"] > thresh)
                
            #print(f"count_regions = {count_regions}")
            #map_img[c//step, y//step] = np.array([c, y, 0])
            map_img[c//step, k//step] = colors[rmm(count_regions, 0, len(colors)-1)]

    return map_img, images

map_img, images = get_images()

### Interactive cell #1

In [ ]:
%matplotlib widget
plt.close("all")

fig, (ax1, ax2) = plt.subplots(1, 2)
im1 = ax1.imshow(map_img)
ax1.set_xlabel("Black(min)")
ax1.set_ylabel("Cian(max)")
secondary_image = images[0][0]
im2 = ax2.imshow(secondary_image)

def on_mouse_move(event):
    if event.inaxes == ax1:
        # Get the pixel coordinates
        i, j = int(event.ydata + 0.5), int(event.xdata + 0.5)
        #print(i, j)

        if 0 <= i < map_img.shape[0] and 0 <= j < map_img.shape[1]:
            # Generate and update the secondary image
            im2.set_data(images[i][j])
            ax2.set_title(f"Pixel ({i}, {j})")
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_mouse_move)

plt.show()

In [ ]:
%matplotlib inline

image_name = "ML-tste_original_test2"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage = cv2.imread(path)
plt.close("all")
plt.imshow(inputImage[:,:,::-1])
plt.show()

# BGR to CMKY:
inputImageCMYK = colorspace.bgr2cmyk(inputImage)

In [ ]:
def get_images():

    thresh = 100
    step = 4

    map_img = np.zeros((256//step, 256//step, 3), dtype=np.uint8)

    colors = [
        np.array([0, 0, 0]),
        np.array([0, 0, 255]),
        np.array([255, 0, 0]),
        np.array([255, 0, 255]),
        np.array([0, 255, 255]),
        np.array([255, 255, 0]),
        np.array([0, 255, 0]),
        np.array([255, 255, 255]),
        ]

    from tqdm import tqdm

    images = [[None for _ in range(256//step)] for _ in range(256//step)]

    for c in tqdm([*range(0, 256, step)]):
        for k in range(0, 256, step):
            binaryImage = cv2.inRange(
                inputImageCMYK,
                ( c,   0,   0,   0),
                (255, 255,  64,  k))
                #(255, 255,  64, 196))
            
            label_img = label(binaryImage)
            regions = regionprops(label_img)
            df = regions_df(regions, label_img)
            
            bin_image_4 = np.zeros((*label_img.shape, 3), dtype=np.uint8)
            id_region = 0
            for it, region in enumerate(regions):
                #display(df)
                if df["area"].iloc[it] > thresh:
                    id_region += 1
                    #color_value = np.array([255, 255, 255])
                    color_value = colors[rmm(id_region, 0, len(colors)-1)]
                    bin_image_4[region.coords.T[0], region.coords.T[1]] = color_value
            images[c//step][k//step] = bin_image_4
            
            # plt.imshow(binaryImage, cmap='gray')
            # plt.show()
            
            if len(regions) == 0:
                count_regions = 0
            else:
                count_regions = sum(df["area"] > thresh)
                
            #print(f"count_regions = {count_regions}")
            #map_img[c//step, y//step] = np.array([c, y, 0])
            map_img[c//step, k//step] = colors[rmm(count_regions, 0, len(colors)-1)]

    return map_img, images

map_img, images = get_images()

### Interactive cell #2

In [ ]:
%matplotlib widget

fig, (ax1, ax2) = plt.subplots(1, 2)
im1 = ax1.imshow(map_img)
ax1.set_xlabel("Black(max)")
ax1.set_ylabel("Cian(min)")
secondary_image = images[0][0]
im2 = ax2.imshow(secondary_image, cmap="inferno")

def on_mouse_move(event):
    if event.inaxes == ax1:
        # Get the pixel coordinates
        i, j = int(event.ydata + 0.5), int(event.xdata + 0.5)
        #print(i, j)

        if 0 <= i < map_img.shape[0] and 0 <= j < map_img.shape[1]:
            # Generate and update the secondary image
            im2.set_data(images[i][j])
            ax2.set_title(f"Pixel ({i}, {j})")
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_mouse_move)

plt.show()